In [ ]:
from glob import glob
import json
from fastprogress import progress_bar

root_path = 'path/to/data/'

folders = glob(f'{root_path}frames/C*/S*/C*S*A0D*')

markers_dict = {}
for folder in progress_bar(folders):
    files = glob(folder + '/*.jpg')
    middle = len(files) // 2
    frame = files[middle]
    subject = frame.split('/')[-3]
    camera = frame.split('/')[-4]
    if subject not in markers_dict:
        markers_dict[subject] = {}
    if camera not in markers_dict[subject]:
        markers_dict[subject][camera] = {}
        markers_dict[subject][camera]['path'] = frame.replace(root_path, '')

with open(f'{root_path}data/markers.json', 'w') as f:
    json.dump(markers_dict, f, indent=4)

In [ ]:
with open(f'{root_path}data/markers_points.json', 'r') as f:
    markers = json.load(f)
keys = list(markers.keys())
subject_nums = [int(key.replace('S', '')) for key in keys if 'points' in markers[key]['C1']]
subject_nums = sorted(subject_nums)
map_nums = {}
cur_num = 1
for i in range(1,29):
    if i in subject_nums:
        cur_num = i
    map_nums[f'S{i}'] = f'S{cur_num}'

map_nums

In [ ]:
import json
import pandas as pd

# Load data
try:
    with open(f'{root_path}data/stomp_refine.json', 'r') as f:
        stomp_data = json.load(f)
except FileNotFoundError:
    stomp_data = {}

combinations_done = list(stomp_data.keys())

df = pd.read_csv(f'{root_path}data/stomps_C4.csv')
df = df[~df['combination'].isin(combinations_done)]

df['first_idx'] = pd.to_numeric(df['first_idx'], errors='coerce').astype('Int64')
df['last_idx'] = pd.to_numeric(df['last_idx'], errors='coerce').astype('Int64')

combinations = df['combination'].tolist()
first_idxs = df['first_idx'].tolist()
last_idxs = df['last_idx'].tolist()

for combination, first_idx, last_idx in zip(combinations, first_idxs, last_idxs):
    stomp_data[combination] = {'first_idx': None, 'last_idx': None}

with open(f'{root_path}data/stomp_refine_manual.json', 'w') as f:
    json.dump(stomp_data, f, indent=4)

In [ ]:
from glob import glob

files = glob(f'{root_path}data/raw_3d_data/**/Wand*.csv')

subjects = [int(file.split('/')[-2][1:]) for file in files]
subjects = sorted(subjects)
print(subjects)
map_nums = {}
cur_num = 16
for i in range(1,29):
    if i in subjects:
        cur_num = i
    map_nums[i] = cur_num

map_nums_3d = map_nums.copy()
map_nums_3d

In [ ]:
import json
import cv2
import os
from tqdm import tqdm
os.makedirs(f'{root_path}data/markers/', exist_ok=True)

with open(f'{root_path}data/markers_points.json', 'r') as f:
    markers = json.load(f)

num_keys = list(markers.keys())
nums = 6*len(num_keys)

pbar = tqdm(total=nums)

for subject, dict1 in markers.items():
    for camera, dict2 in dict1.items():
        if 'points' in dict2:
            path = dict2['path']
            points = dict2['points']
            save_path = f'{root_path}data/markers/' + path.split('/')[-1]
            img = cv2.imread(root_path + path)
            for i, point in enumerate(points):
                cv2.circle(img, (int(point[0]), int(point[1])), 5, (0, 255, 0), -1)
                #plot idx
                cv2.putText(img, str(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imwrite(save_path, img)
        pbar.update(1)

In [ ]:
import json
import cv2
import os
from tqdm import tqdm
os.makedirs(f'{root_path}data/markers_proj/', exist_ok=True)

with open(f'{root_path}data/markers_points.json', 'r') as f:
    markers = json.load(f)

num_keys = list(markers.keys())
nums = 6*len(num_keys)

pbar = tqdm(total=nums)

for subject, dict1 in markers.items():
    for camera, dict2 in dict1.items():
        if 'points' in dict2:
            path = dict2['path']
            points = dict2['points']
            save_path = f'{root_path}data/markers_proj/' + path.split('/')[-1]
            with open(f'{root_path}internal_data/{camera}/{subject}/{subject}A1D1.json', 'r') as f:
                data = json.load(f)
                frame = data[0]
                wand = frame['wand_xy']
            img = cv2.imread(root_path + path)
            for i, point in enumerate(points):
                cv2.circle(img, (int(point[0]), int(point[1])), 5, (0, 255, 0), -1)
            for i, point in enumerate(wand):
                cv2.circle(img, (int(point[0]), int(point[1])), 5, (255, 0, 0), -1)
            cv2.imwrite(save_path, img)
        pbar.update(1)

In [35]:
with open(f'{root_path}data/markers_ids.json', 'r') as f:
    markers_ids = json.load(f)
with open(f'{root_path}data/markers_init.json', 'r') as f:
    markers_init = json.load(f)

keys = list(markers_ids.keys())

new_dict = {}
for key in keys:
    cam = key.split('S')[0]
    subject = f"S{key.split('S')[1].split('A')[0]}"
    combination = key.split('_')[0]
    path = f'frames/{cam}/{subject}/{combination}/{key}'
    points = markers_init[key]
    ids = markers_ids[key]
    if len(ids) != len(points):
        print(f"Error: {key}")
    if subject not in new_dict:
        new_dict[subject] = {}
    if cam not in new_dict[subject]:
        new_dict[subject][cam] = {}
    new_dict[subject][cam] = {'path': path, 'points': points, 'ids': ids}

with open(f'{root_path}data/markers_finetune.json', 'w') as f:
    json.dump(new_dict, f, indent=4)

In [27]:
import numpy as np
import cv2
from scipy import linalg
import pandas as pd
import matplotlib.pyplot as plt
from fastprogress import progress_bar
import json
from glob import glob
import os


df_cam_matrix = pd.read_csv(f'{root_path}calibration/external/csvs/external_calibrations_w_errors_best.csv')

matrix_subjects = df_cam_matrix['subject'].unique().tolist()
matrix_subjects = sorted(matrix_subjects)
matrix_subjects = [int(subj.replace('S', '')) for subj in matrix_subjects]
map_matrix = {}
cur_num = 1
for i in range(1,29):
    if i in matrix_subjects:
        cur_num = i
    map_matrix[f'S{i}'] = f'S{cur_num}'

with open(f'{root_path}data/markers_points.json', 'r') as f:
    markers = json.load(f)
keys = list(markers.keys())
subject_nums = [int(key.replace('S', '')) for key in keys if 'points' in markers[key]['C1']]
subject_nums = sorted(subject_nums)
map_nums = {}
cur_num = 1
for i in range(1,29):
    if i in subject_nums:
        cur_num = i
    map_nums[f'S{i}'] = f'S{cur_num}'

c1_data = []
c2_data = []
c3_data = []
c4_data = []
c5_data = []
c6_data = []

files = glob(f'{root_path}WCS/S17/S17A1D1.json')
print(len(files))
for file in progress_bar(files):
    c1_data = []
    c2_data = []
    c3_data = []
    c4_data = []
    c5_data = []
    c6_data = []
    subject = file.split('/')[-2]
    combintation = file.split('/')[-1].replace('.json', '')
    print(f'Processing {subject} {combintation}')
    with open(file, 'r') as f:
        frames = json.load(f)
    frames = frames[:1]
    print(len(frames))
    for frame in progress_bar(frames):
        xyz = frame['xyz']
        xyz_wand = frame['xyz_wand']
        point_ids = frame['point_ids']
        floor_idx = point_ids.index('CentreOfMassFloor')
        # floor_y = xyz[floor_idx][1]
        points_skel = np.array(xyz)
        wcs_skel = []
        wcs_skel.append(points_skel)
        wcs_skel = np.array(wcs_skel)
        wcs_skel = wcs_skel.reshape(wcs_skel.shape[1], wcs_skel.shape[2])
        wcs_skel[:, 0] = wcs_skel[:, 0]/60
        wcs_skel[:, 1] = wcs_skel[:, 1]/60
        wcs_skel[:, 2] = wcs_skel[:, 2]/60

        wcs_wand = []
        wcs_wand.append(xyz_wand)
        wcs_wand = np.array(wcs_wand)
        wcs_wand = wcs_wand.reshape(wcs_wand.shape[1], wcs_wand.shape[2])
        wcs_wand[:, 0] = wcs_wand[:, 0]/60
        wcs_wand[:, 1] = wcs_wand[:, 1]/60
        wcs_wand[:, 2] = wcs_wand[:, 2]/60

        for main in [1, 6]:
            if main == 1:
                pairs = [2, 3]
            else:
                pairs = [4, 5]

            df_temp = df_cam_matrix[(df_cam_matrix['main'] == main) & (df_cam_matrix['pair'] == pairs[0]) & (df_cam_matrix['subject'] == map_matrix[subject])].copy()
            path1 = df_temp['path'].values[0]
            df_temp = df_cam_matrix[(df_cam_matrix['main'] == main) & (df_cam_matrix['pair'] == pairs[1]) & (df_cam_matrix['subject'] == map_matrix[subject])].copy()
            path2 = df_temp['path'].values[0]
            data = np.load(path1)
            data2 = np.load(path2)

            R1 = np.eye(3)  
            T1 = np.array([0, 0, 0])  
            R2 = data['R']
            T2 = data['T']
            T2 = np.array([T2[0][0], T2[1][0], T2[2][0]]).reshape((3,1))
            R3 = data2['R']
            T3 = data2['T']
            T3 = np.array([T3[0][0], T3[1][0], T3[2][0]]).reshape((3,1))

            cameraMatrix1 = data['mtx_A']
            cameraMatrix2 = data['mtx_B']
            cameraMatrix3 = data2['mtx_B']

            RT1 = np.concatenate([R1, [[0],[0],[0]]], axis = -1)
            P1 = cameraMatrix1 @ RT1
            RT2 = np.concatenate([R2, T2], axis = -1)
            P2 = cameraMatrix2 @ RT2
            RT3 = np.concatenate([R3, T3], axis = -1)
            P3 = cameraMatrix3 @ RT3

            Ps = [P1, P2, P3]

            def triangulate_multiple_points(imgpoints_Origin, imgpoints_Pair1, imgpoints_Pair2, projMatr_Origin, projMatr_Pair1, projMatr_Pair2):
                points_3d_list = []
                for points_origin, points_pair1, points_pair2 in zip(imgpoints_Origin, imgpoints_Pair1, imgpoints_Pair2):
                    point_origin = np.array(points_origin)
                    point_pair1 = np.array(points_pair1)
                    point_pair2 = np.array(points_pair2)

                    A = [point_origin[1]*projMatr_Origin[2,:] - projMatr_Origin[1,:],
                        projMatr_Origin[0,:] - point_origin[0]*projMatr_Origin[2,:],
                        point_pair1[1]*projMatr_Pair1[2,:] - projMatr_Pair1[1,:],
                        projMatr_Pair1[0,:] - point_pair1[0]*projMatr_Pair1[2,:],
                        point_pair2[1]*projMatr_Pair2[2,:] - projMatr_Pair2[1,:],
                        projMatr_Pair2[0,:] - point_pair2[0]*projMatr_Pair2[2,:],
                        ]

                    A = np.array(A) 
                    B = A.transpose() @ A
                    U, s, Vh = linalg.svd(B, full_matrices = False)
                    
                    points4D_homogeneous = Vh[-1,0:3]/Vh[-1,3]
                    points_3d_list.append(points4D_homogeneous[:3].T)

                return points_3d_list

            all_cam_points = [
                markers[map_nums[subject]][f'C{main}']['points'],
                markers[map_nums[subject]][f'C{pairs[0]}']['points'],
                markers[map_nums[subject]][f'C{pairs[1]}']['points']
            ]

            all_results = []

            all_results.append(triangulate_multiple_points(all_cam_points[0], all_cam_points[1], all_cam_points[2], Ps[0], Ps[1], Ps[2]))
            R1_initial = R1.copy()
            T1_initial = T1.reshape((3,1)).copy()
            R2_initial = R2.copy()
            T2_initial = T2.reshape((3,1)).copy()
            R3_initial = R3.copy()
            T3_initial = T3.reshape((3,1)).copy()
            points_initial = (all_results[0].copy())
            points_initial = np.array(points_initial)
            points = np.array(points_initial)

            x_direction = points[2] - points[1] 
            z_direction = points[4] - points[1]  
            x_direction /= np.linalg.norm(x_direction)
            z_direction /= np.linalg.norm(z_direction)
            y_direction = np.cross(x_direction, z_direction)

            R_wcs = np.column_stack((x_direction, y_direction, z_direction))
            M = np.column_stack((R_wcs, points[1]))
            M = np.row_stack((M, np.array([0, 0, 0, 1])))
            M_inv = np.linalg.inv(M)

            rotation_matrix = np.column_stack((x_direction, y_direction, z_direction))

            R_initial = rotation_matrix.copy()

            # plot_graph(points[:, 0], points[:, 1], points[:, 2], T1_initial, T2_initial, T3_initial, R1_initial, R2_initial, R3_initial, R_initial)
            import plotly.graph_objects as go

            x = points[:, 0]
            y = points[:, 1]
            z = points[:, 2]

            def plot_vector(fig, start, direction, name, color):
                end = start + direction
                fig.add_trace(go.Scatter3d(x=[start[0], end[0]], y=[start[1], end[1]], z=[start[2], end[2]],
                                        mode='lines+text', line=dict(color=color, width=5)))

            # Create interactive 3D scatter plot
            fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=8))])
            fig.add_trace(go.Scatter3d(x=T1_initial[0], y=T1_initial[1], z=T1_initial[2], mode='markers', marker=dict(size=10, color='red'), name='1'))
            fig.add_trace(go.Scatter3d(x=T2_initial[0], y=T2_initial[1], z=T2_initial[2], mode='markers', marker=dict(size=10, color='green'), name='2'))
            fig.add_trace(go.Scatter3d(x=T3_initial[0], y=T3_initial[1], z=T3_initial[2], mode='markers', marker=dict(size=10, color='yellow'), name='3'))

            colors = ['red', 'green', 'blue']  # Colors for the x, y, z axis directions
            for i in range(3):
                direction = R1_initial[:, i]*10
                plot_vector(fig, T1_initial.flatten(), direction, f'I{i+1}', colors[i])
            for i in range(3):
                direction = R2_initial[:, i]*10
                plot_vector(fig, T2_initial.flatten(), direction, f'R{i+1}', colors[i])
            for i in range(3):
                direction = R3_initial[:, i]*10
                plot_vector(fig, T3_initial.flatten(), direction, f'R{i+1}', colors[i])
            for i in range(3):
                direction = R_initial[:, i]*10
                plot_vector(fig, points[1], direction, f'I{i+1}', colors[i])

            fig.update_layout(scene=dict(
                                xaxis_title='X',
                                yaxis_title='Y',
                                zaxis_title='Z'),
                            width=700,
                            margin=dict(r=20, b=10, l=10, t=10))
            fig.show()

            def projectInternal(M, xyz):
                xyz = np.array([[xyz[0], xyz[1], xyz[2], 1]], dtype=object).T
                internal_xyz = M @ xyz
                internal_xyz[:3].flatten()
                internal_xyz = np.squeeze(internal_xyz)
                internal_xyz =np.squeeze(internal_xyz)
                return internal_xyz[:3]
            
            # calculate C1 internal 3D points
            c1_wand = np.array([projectInternal(M, p) for p in wcs_wand])
            c1_skel = np.array([projectInternal(M, p) for p in wcs_skel])
            # floor_y = c1_skel[floor_idx][1]
            # c1_skel[:,1] += np.abs(floor_y-(np.mean(points[:, 1])))
            order = [1, 0, 2, 3, 4] 
            c1_wand = c1_wand[order, :]
            diff = points - c1_wand
            y_avg_wand = np.mean(c1_wand[:, 1])
            y_avg_points = np.mean(points[:, 1])
            # y_diff = np.abs(y_avg_points - y_avg_wand)
            y_diff = y_avg_points - y_avg_wand
            x_avg_wand = np.mean(c1_wand[:, 0])
            x_avg_points = np.mean(points[:, 0])
            # x_diff = np.abs(x_avg_points - x_avg_wand)
            x_diff = x_avg_points - x_avg_wand
            z_avg_wand = np.mean(c1_wand[:, 2])
            z_avg_points = np.mean(points[:, 2])
            # z_diff = np.abs(z_avg_points - z_avg_wand)
            z_diff = z_avg_points - z_avg_wand
            c1_wand[:, 1] += y_diff
            c1_skel[:, 1] += y_diff
            c1_wand[:, 0] += x_diff
            c1_skel[:, 0] += x_diff
            c1_wand[:, 2] += z_diff
            c1_skel[:, 2] += z_diff

            def projectPixel(cam_params, xyz):
                K = cam_params['mtx']
                a = np.array([[0, 0, 0]]).T
                xyz = np.array([[xyz[0], xyz[1], xyz[2], 1]], dtype=object).T
                M = np.hstack((K, a))
                image_xy = M @ xyz
                image_xy[:3].flatten()
                image_xy /= image_xy[2]
                image_xy = np.squeeze(image_xy)
                return image_xy[0:2]

            cam_params={}
            cam_params['C2']={}
            cam_params['C2']['R'] = R1_initial
            cam_params['C2']['T'] = T1_initial
            cam_params['C2']['mtx'] = cameraMatrix1

            # calculate C1 pixel points
            c1_pixel_points = np.array([projectPixel(cam_params['C2'], p) for p in points])
            c1_pixel_skel = np.array([projectPixel(cam_params['C2'], p) for p in c1_skel])

            data_dict = {
                'point_ids': point_ids,
                'xyz': c1_skel.tolist(),
                'xy': c1_pixel_skel.tolist(),
                'wand_xyz': points.tolist(),
                'wand_xy': c1_pixel_points.tolist()                                            
            }
            print(f'Main: {main}')
            fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=5))])
            fig.add_trace(go.Scatter3d(x=T1_initial[0], y=T1_initial[1], z=T1_initial[2], mode='markers', marker=dict(size=10, color='red'), name='1'))
            fig.add_trace(go.Scatter3d(x=T2_initial[0], y=T2_initial[1], z=T2_initial[2], mode='markers', marker=dict(size=10, color='green'), name='2'))
            fig.add_trace(go.Scatter3d(x=T3_initial[0], y=T3_initial[1], z=T3_initial[2], mode='markers', marker=dict(size=10, color='yellow'), name='3'))
            fig.add_trace(go.Scatter3d(x=c1_wand[:, 0], y=c1_wand[:, 1], z=c1_wand[:, 2], mode='markers', marker=dict(size=4, color='black'), name='wand'))
            colors = ['red', 'green', 'blue']  # Colors for the x, y, z axis directions
            for i in range(3):
                direction = R1_initial[:, i]*10
                plot_vector(fig, T1_initial.flatten(), direction, f'I{i+1}', colors[i])
            for i in range(3):
                direction = R2_initial[:, i]*10
                plot_vector(fig, T2_initial.flatten(), direction, f'R{i+1}', colors[i])
            for i in range(3):
                direction = R3_initial[:, i]*10
                plot_vector(fig, T3_initial.flatten(), direction, f'R{i+1}', colors[i])
            for i in range(3):
                direction = R_initial[:, i]*10
                plot_vector(fig, points[1], direction, f'I{i+1}', colors[i])

            fig.update_layout(scene=dict(
                                xaxis_title='X',
                                yaxis_title='Y',
                                zaxis_title='Z'),
                            width=700,
                            margin=dict(r=20, b=10, l=10, t=10))
            fig.show()

            if main == 1:
                c1_data.append(data_dict)
            else:
                c6_data.append(data_dict)

            def projectInternal(cam_params, xyz):
                R = cam_params['R']
                t = cam_params['T']
                a = np.array([[0, 0, 0]])
                b = np.array([[1]])
                xyz = np.array([[xyz[0], xyz[1], xyz[2], 1]], dtype=object).T
                M = np.vstack((np.hstack((R, t)), np.hstack((a, b))))
                internal_xyz = M @ xyz
                internal_xyz[:3].flatten()
                internal_xyz = np.squeeze(internal_xyz)
                internal_xyz =np.squeeze(internal_xyz)
                return internal_xyz[:3]
                # return xyz
                
            cam_params={}
            cam_params['C2']={}
            cam_params['C2']['R'] = R2_initial
            cam_params['C2']['T'] = T2_initial
            cam_params['C2']['mtx'] = cameraMatrix2

            # calculate C2 internal 3D points
            c2_3d_points = np.array([projectInternal(cam_params['C2'], p) for p in points])
            c2_3d_skel = np.array([projectInternal(cam_params['C2'], p) for p in c1_skel])   
            
            # calculate C2 pixel points
            c2_pixel_points = np.array([projectPixel(cam_params['C2'], p) for p in c2_3d_points])
            c2_pixel_skel = np.array([projectPixel(cam_params['C2'], p) for p in c2_3d_skel]) 

            data_dict = {
                'point_ids': point_ids,
                'xyz': c2_3d_skel.tolist(),
                'xy': c2_pixel_skel.tolist(),
                'wand_xyz': c2_3d_points.tolist(),
                'wand_xy': c2_pixel_points.tolist()                                            
            }

            c2_3d_points = np.array(c2_3d_points, dtype=float)

            x_direction = c2_3d_points[2] - c2_3d_points[1] 
            z_direction = c2_3d_points[4] - c2_3d_points[1]  
            x_direction /= np.linalg.norm(x_direction)
            z_direction /= np.linalg.norm(z_direction)
            y_direction = np.cross(x_direction, z_direction)

            R_temp = np.column_stack((x_direction, y_direction, z_direction))

            x = c2_3d_points[:, 0]
            y = c2_3d_points[:, 1]
            z = c2_3d_points[:, 2]

            T1_temp = np.array([projectInternal(cam_params['C2'], p) for p in T1_initial.T])
            T2_temp = np.array([projectInternal(cam_params['C2'], p) for p in T2_initial.T])
            T3_temp = np.array([projectInternal(cam_params['C2'], p) for p in T3_initial.T])
            wand_temp = np.array([projectInternal(cam_params['C2'], p) for p in c1_wand])
            print(f'Pair: {pairs[0]}')
            fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=5))])
            fig.add_trace(go.Scatter3d(x=T1_temp.T[0], y=T1_temp.T[1], z=T1_temp.T[2], mode='markers', marker=dict(size=10, color='red'), name='1'))
            fig.add_trace(go.Scatter3d(x=T2_temp.T[0], y=T2_temp.T[1], z=T2_temp.T[2], mode='markers', marker=dict(size=10, color='green'), name='2'))
            fig.add_trace(go.Scatter3d(x=T3_temp.T[0], y=T3_temp.T[1], z=T3_temp.T[2], mode='markers', marker=dict(size=10, color='yellow'), name='3'))
            fig.add_trace(go.Scatter3d(x=wand_temp[:, 0], y=wand_temp[:, 1], z=wand_temp[:, 2], mode='markers', marker=dict(size=4, color='black'), name='wand'))
            colors = ['red', 'green', 'blue']  # Colors for the x, y, z axis directions

            for i in range(3):
                direction = R_temp[:, i]*10
                plot_vector(fig, c2_3d_points[1], direction, f'I{i+1}', colors[i])

            fig.update_layout(scene=dict(
                                xaxis_title='X',
                                yaxis_title='Y',
                                zaxis_title='Z'),
                            width=700,
                            margin=dict(r=20, b=10, l=10, t=10))
            fig.show()
        
            if main == 1:
                c2_data.append(data_dict)
            else:
                c4_data.append(data_dict)

            cam_params={}
            cam_params['C2']={}
            cam_params['C2']['R'] = R3_initial
            cam_params['C2']['T'] = T3_initial
            cam_params['C2']['mtx'] = cameraMatrix3

            # calculate C3 internal 3D points
            c2_3d_points = np.array([projectInternal(cam_params['C2'], p) for p in points])
            c2_3d_skel = np.array([projectInternal(cam_params['C2'], p) for p in c1_skel])  

            # calculate C3 pixel points
            c2_pixel_points = np.array([projectPixel(cam_params['C2'], p) for p in c2_3d_points])
            c2_pixel_skel = np.array([projectPixel(cam_params['C2'], p) for p in c2_3d_skel])

            data_dict = {
                'point_ids': point_ids,
                'xyz': c2_3d_skel.tolist(),
                'xy': c2_pixel_skel.tolist(),
                'wand_xyz': c2_3d_points.tolist(),
                'wand_xy': c2_pixel_points.tolist()                                            
            }

            c2_3d_points = np.array(c2_3d_points, dtype=float)

            x_direction = c2_3d_points[2] - c2_3d_points[1] 
            z_direction = c2_3d_points[4] - c2_3d_points[1]  
            x_direction /= np.linalg.norm(x_direction)
            z_direction /= np.linalg.norm(z_direction)
            y_direction = np.cross(x_direction, z_direction)

            R_temp = np.column_stack((x_direction, y_direction, z_direction))

            x = c2_3d_points[:, 0]
            y = c2_3d_points[:, 1]
            z = c2_3d_points[:, 2]

            T1_temp = np.array([projectInternal(cam_params['C2'], p) for p in T1_initial.T])
            T2_temp = np.array([projectInternal(cam_params['C2'], p) for p in T2_initial.T])
            T3_temp = np.array([projectInternal(cam_params['C2'], p) for p in T3_initial.T])
            wand_temp = np.array([projectInternal(cam_params['C2'], p) for p in c1_wand])
            print(f'Pair: {pairs[1]}')  
            fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=5))])
            fig.add_trace(go.Scatter3d(x=T1_temp.T[0], y=T1_temp.T[1], z=T1_temp.T[2], mode='markers', marker=dict(size=10, color='red'), name='1'))
            fig.add_trace(go.Scatter3d(x=T2_temp.T[0], y=T2_temp.T[1], z=T2_temp.T[2], mode='markers', marker=dict(size=10, color='green'), name='2'))
            fig.add_trace(go.Scatter3d(x=T3_temp.T[0], y=T3_temp.T[1], z=T3_temp.T[2], mode='markers', marker=dict(size=10, color='yellow'), name='3'))
            fig.add_trace(go.Scatter3d(x=wand_temp[:, 0], y=wand_temp[:, 1], z=wand_temp[:, 2], mode='markers', marker=dict(size=4, color='black'), name='wand'))
            colors = ['red', 'green', 'blue']  # Colors for the x, y, z axis directions

            for i in range(3):
                direction = R_temp[:, i]*10
                plot_vector(fig, c2_3d_points[1], direction, f'I{i+1}', colors[i])

            fig.update_layout(scene=dict(
                                xaxis_title='X',
                                yaxis_title='Y',
                                zaxis_title='Z'),
                            width=700,
                            margin=dict(r=20, b=10, l=10, t=10))
            fig.show()

            if main == 1:
                c3_data.append(data_dict)
            else:
                c5_data.append(data_dict)

1


Processing S17 S17A1D1
1


Main: 1


Pair: 2


Pair: 3


Main: 6


Pair: 4


Pair: 5


In [32]:
df_intrinsic = pd.read_csv(f'{root_path}calibration/internal/csvs/internal_calibration_best.csv')
matrix_subjects = df_intrinsic['subject'].unique().tolist()
matrix_subjects = sorted(matrix_subjects)
map_matrix = {}
cur_num = 1
for i in range(1,29):
    if i in matrix_subjects:
        cur_num = i
    map_matrix[f'S{i}'] = f'S{cur_num}'
map_nums = map_matrix
map_nums

{'S1': 'S1',
 'S2': 'S1',
 'S3': 'S1',
 'S4': 'S1',
 'S5': 'S1',
 'S6': 'S6',
 'S7': 'S6',
 'S8': 'S6',
 'S9': 'S6',
 'S10': 'S6',
 'S11': 'S11',
 'S12': 'S11',
 'S13': 'S11',
 'S14': 'S11',
 'S15': 'S11',
 'S16': 'S11',
 'S17': 'S17',
 'S18': 'S17',
 'S19': 'S17',
 'S20': 'S17',
 'S21': 'S17',
 'S22': 'S17',
 'S23': 'S23',
 'S24': 'S23',
 'S25': 'S23',
 'S26': 'S23',
 'S27': 'S23',
 'S28': 'S23'}